In [36]:
import pandas as pd
from src.criteria import conditions, staff
from src.database import airports, offices_list, companies_design, companies_1m
from src.score import add_score, total_score

## Initial considerations:

1. All the workers are considered equal (including the dog)
2. The closest is a location the better
3. The more locations that meet the criteria the better until a maximum number where more locations do not give an extra value

With this consideration a formula was generated

### Score formula

![Score Formula](./img/score-function.png)

**condition(x)** &rarr; Conditions score from x=1 to x=9.
**ocu** &rarr; Number of occurrences for a given condition to a maximum "max(ocu)".
**max(ocu)** &rarr; Maximum number of locations that are considered enough to satisfy a given condition.
**dis** &rarr; Distance between two locations.
**min(dis)** &rarr; Minimum distance from the location and places that meet the condition.
**min(dis)** &rarr; Maximum distance from the location that can be situated a place that meets the condition.
**people** &rarr; Number of people in the company that satify a given condition
**max(score)** &rarr; The maximum score possible of the sum of all conditions score

![Total](./img/total-score-formula.png)


## Process to Find the Best Locations:

### 1. Definig Conditions Parameters:

In [2]:
pd.DataFrame(conditions)

,1,2,3,4,5,6,7,8,9
max_num,10,5,20,1,1,30,2,1,1
max_dist,5,3,20,2,50,3,2,10,4
people,20,26,15,10,20,87,1,1,1
description,Designers like to go to design talks and share...,30% of the company staff have at least 1 child.,Developers like to be near successful tech sta...,Executives like Starbucks A LOT. Ensure there'...,Account managers need to travel a lot.,"Everyone in the company is between 25 and 40, ...",The CEO is vegan.,"If you want to make the maintenance guy happy,...",The office dog Dobby needs a hairdresser every...
source,Crunchbase,Foursquare,Crunchbase,Foursquare,OurAirports,Foursquare,Foursquare,Foursquare,Foursquare
categories,"[tags_list, description, name]","[12055, 12056, 12057, 12058, 12059, 12060, 120...",total_money_raised,[13032],None,"[13003, 10032]",[13377],[18008],[11134]
query,design,None,"More than 1,000,000",Starbucks,None,None,None,None,None


In [3]:
pd.Series(staff)

design      20
uiux         5
frontend    10
data        15
backend      5
account     20
maint        1
exec        10
ceo          1
dog          1
dtype: int64

### 2. Conditions that doesn't require Forsquare

Conditions 1, 2, and 3 do not require the use of Foursquare API that limits the number of requests. 

For this reason, these conditions are evaluated to make a first filter of the best candidate offices.

The process is done as follows:

    1) Upload Crunchbase database to MongoDB
    2) Query MongoDB
    3) Clean queries to generate a list of all offices locations (10,743 entries)
    4) Clean queries to generate a list of offices from Design companies (498 entries)
    5) Clean queries to generate a list of offices from companies that raised more than 1 million USD (3,253 entries)
    6) Download and clean the database of airports from OurAirports (8,636 entries)

In [4]:
# Creating databases
offices = offices_list()
airports = airports()
companies_design = companies_design()
companies_1m = companies_1m()

In [5]:
offices.head()

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,AdventNet,USA,Pleasanton,37.692934,-121.904945
3,Zoho,USA,Pleasanton,37.692934,-121.904945
4,Digg,USA,San Francisco,37.764726,-122.394523


In [6]:
airports.head()

,name,iata_code,iso_country,latitude,longitude
0,Utirik Airport,UTK,MH,11.222000,169.852005
1,Ocean Reef Club Airport,OCA,US,25.325399,-80.274803
2,Pilot Station Airport,PQS,US,61.934601,-162.899994
3,Crested Butte Airpark,CSE,US,38.851918,-106.928341
4,LBJ Ranch Airport,JCY,US,30.251801,-98.622498


In [22]:
companies_design.head()

,name,country_code,city,latitude,longitude
0,Digg,USA,San Francisco,37.764726,-122.394523
1,Adobe Systems,USA,San Jose,37.320052,-121.877636
2,Negonation,ESP,,40.463667,-3.749220
3,GENWI,USA,Los Altos,33.817100,-111.903500
4,BrightQube,USA,Carlsbad,33.095680,-117.244034


In [8]:
companies_1m.head()

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,Digg,USA,San Francisco,37.764726,-122.394523
3,Facebook,USA,Menlo Park,37.416050,-122.151801
4,Facebook,IRL,Dublin,53.344104,-6.267494


### Condition 1


```python
offices2 = add_score(offices, 1)
```

In [10]:
offices2.head()

,name,country_code,city,latitude,longitude,condition1_results,condition1_score
0,Wetpaint,USA,Seattle,47.603122,-122.333253,"[{'Unnamed: 0': 85, 'name': 'Fixie', 'country_...",0.067506
1,Wetpaint,USA,New York,40.723731,-73.996431,"[{'Unnamed: 0': 82, 'name': 'IDEO', 'country_c...",0.080295
2,AdventNet,USA,Pleasanton,37.692934,-121.904945,[],0.000000
3,Zoho,USA,Pleasanton,37.692934,-121.904945,[],0.000000
4,Digg,USA,San Francisco,37.764726,-122.394523,"[{'Unnamed: 0': 126, 'name': 'Ecolect', 'count...",0.086383


### Condition 3

```python
offices2 = add_score(offices2, 3)
```

In [34]:
offices2.head()

,Unnamed: 0,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,condition5_score
0,0,Wetpaint,USA,Seattle,47.603122,-122.333253,"[{'Unnamed: 0': 241, 'name': 'Random Computers...",0.075531,"[{'Unnamed: 0': 2485, 'name': 'Z2Live', 'count...",0.082338,"[{'Unnamed: 0': 2610, 'name': 'Boeing Field Ki...",0.099938
1,1,Wetpaint,USA,New York,40.723731,-73.996431,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943
2,2,AdventNet,USA,Pleasanton,37.692934,-121.904945,[],0.000000,"[{'Unnamed: 0': 758, 'name': 'Workday', 'count...",0.079580,"[{'Unnamed: 0': 3346, 'name': 'Livermore Munic...",0.099077
3,3,Zoho,USA,Pleasanton,37.692934,-121.904945,[],0.000000,"[{'Unnamed: 0': 758, 'name': 'Workday', 'count...",0.079580,"[{'Unnamed: 0': 3346, 'name': 'Livermore Munic...",0.099077
4,4,Digg,USA,San Francisco,37.764726,-122.394523,"[{'Unnamed: 0': 13, 'name': '99designs', 'coun...",0.107358,"[{'Unnamed: 0': 2107, 'name': 'Focus', 'countr...",0.082873,"[{'Unnamed: 0': 3187, 'name': 'Hayward Executi...",0.091690


In [14]:
offices2.to_csv('./data/offices_score.csv')

## Condition 5

```python
offices2 = add_score(offices2, 5)
```

In [35]:
offices2.head()

,Unnamed: 0,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,condition5_score
0,0,Wetpaint,USA,Seattle,47.603122,-122.333253,"[{'Unnamed: 0': 241, 'name': 'Random Computers...",0.075531,"[{'Unnamed: 0': 2485, 'name': 'Z2Live', 'count...",0.082338,"[{'Unnamed: 0': 2610, 'name': 'Boeing Field Ki...",0.099938
1,1,Wetpaint,USA,New York,40.723731,-73.996431,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943
2,2,AdventNet,USA,Pleasanton,37.692934,-121.904945,[],0.000000,"[{'Unnamed: 0': 758, 'name': 'Workday', 'count...",0.079580,"[{'Unnamed: 0': 3346, 'name': 'Livermore Munic...",0.099077
3,3,Zoho,USA,Pleasanton,37.692934,-121.904945,[],0.000000,"[{'Unnamed: 0': 758, 'name': 'Workday', 'count...",0.079580,"[{'Unnamed: 0': 3346, 'name': 'Livermore Munic...",0.099077
4,4,Digg,USA,San Francisco,37.764726,-122.394523,"[{'Unnamed: 0': 13, 'name': '99designs', 'coun...",0.107358,"[{'Unnamed: 0': 2107, 'name': 'Focus', 'countr...",0.082873,"[{'Unnamed: 0': 3187, 'name': 'Hayward Executi...",0.091690


In [25]:
# Saving database
# offices2 = pd.read_csv('./data/offices_score.csv')
# Function to apply to a results columns if imported from csv backup
'''
from numpy import nan, log
def fixdf(e):
    
    if type(e) == str:
        e = eval(e)
    return e
'''



### 3. Getting the Top 200 locations for the conditions evaluated

In [67]:
### Getting the provisional total score
offices2 = total_score(offices2)
top_offices = offices2[:200].copy(deep = True)
top_offices.reset_index(drop=True, inplace=True)
top_offices.drop(columns=["Unnamed: 0"], inplace=True)

In [68]:
top_offices.head()

,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,condition5_score,total_score
0,Zoomabet,USA,NY,40.754698,-73.992152,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146
1,Solidea,USA,New York,40.715145,-74.005897,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146
2,Live2Support,USA,Long Island City,40.748876,-73.981579,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146
3,Petciety,USA,New York,40.753207,-73.995886,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146
4,Cowrite,USA,New York,40.767924,-73.983602,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146


### 4. Query FourSquare to Calculate the Total Score of the Top 200

In [73]:
### Conditon 2
top_offices = sc.add_score(top_offices, 2)

In [74]:
top_offices.head()

,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,condition5_score,total_score,condition2_results,condition2_score
0,Zoomabet,USA,NY,40.754698,-73.992152,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.142689
1,Solidea,USA,New York,40.715145,-74.005897,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.138882
2,Live2Support,USA,Long Island City,40.748876,-73.981579,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 11025, 'name': 'Child ...",0.137877
3,Petciety,USA,New York,40.753207,-73.995886,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.135554
4,Cowrite,USA,New York,40.767924,-73.983602,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.142138


In [77]:
### Condition 4
top_offices = sc.add_score(top_offices, 4)

In [78]:
top_offices.head()

,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,condition5_score,total_score,condition2_results,condition2_score,condition4_results,condition4_score
0,Zoomabet,USA,NY,40.754698,-73.992152,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.142689,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.052238
1,Solidea,USA,New York,40.715145,-74.005897,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.138882,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054807
2,Live2Support,USA,Long Island City,40.748876,-73.981579,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 11025, 'name': 'Child ...",0.137877,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054420
3,Petciety,USA,New York,40.753207,-73.995886,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.135554,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054378
4,Cowrite,USA,New York,40.767924,-73.983602,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.142138,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054061


In [79]:
### Condition 6
top_offices = sc.add_score(top_offices, 6)

In [81]:
top_offices.head()

,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,condition5_score,total_score,condition2_results,condition2_score,condition4_results,condition4_score,condition6_results,condition6_score
0,Zoomabet,USA,NY,40.754698,-73.992152,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.142689,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.052238,"[{'categories': [{'id': 13003, 'name': 'Bar', ...",0.307704
1,Solidea,USA,New York,40.715145,-74.005897,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.138882,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054807,"[{'categories': [{'id': 13010, 'name': 'Dive B...",0.271334
2,Live2Support,USA,Long Island City,40.748876,-73.981579,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 11025, 'name': 'Child ...",0.137877,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054420,"[{'categories': [{'id': 13007, 'name': 'Beer G...",0.296729
3,Petciety,USA,New York,40.753207,-73.995886,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.135554,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054378,"[{'categories': [{'id': 13006, 'name': 'Beer B...",0.282870
4,Cowrite,USA,New York,40.767924,-73.983602,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.142138,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054061,"[{'categories': [{'id': 13011, 'name': 'Gay Ba...",0.300735


In [82]:
### Condition 7
top_offices = sc.add_score(top_offices, 7)

In [83]:
top_offices.head()

,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,condition5_score,total_score,condition2_results,condition2_score,condition4_results,condition4_score,condition6_results,condition6_score,condition7_results,condition7_score
0,Zoomabet,USA,NY,40.754698,-73.992152,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.142689,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.052238,"[{'categories': [{'id': 13003, 'name': 'Bar', ...",0.307704,"[{'categories': [{'id': 13068, 'name': 'Americ...",0.005365
1,Solidea,USA,New York,40.715145,-74.005897,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.138882,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054807,"[{'categories': [{'id': 13010, 'name': 'Dive B...",0.271334,"[{'categories': [{'id': 13145, 'name': 'Fast F...",0.005311
2,Live2Support,USA,Long Island City,40.748876,-73.981579,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 11025, 'name': 'Child ...",0.137877,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054420,"[{'categories': [{'id': 13007, 'name': 'Beer G...",0.296729,"[{'categories': [{'id': 13072, 'name': 'Asian ...",0.005315
3,Petciety,USA,New York,40.753207,-73.995886,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.135554,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054378,"[{'categories': [{'id': 13006, 'name': 'Beer B...",0.282870,"[{'categories': [{'id': 13068, 'name': 'Americ...",0.005122
4,Cowrite,USA,New York,40.767924,-73.983602,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",0.101943,0.295146,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.142138,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054061,"[{'categories': [{'id': 13011, 'name': 'Gay Ba...",0.300735,"[{'categories': [{'id': 13065, 'name': 'Restau...",0.005059


In [85]:
### Condition 8
top_offices = sc.add_score(top_offices, 8)

In [86]:
top_offices.head()

,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,...,condition2_results,condition2_score,condition4_results,condition4_score,condition6_results,condition6_score,condition7_results,condition7_score,condition8_results,condition8_score
0,Zoomabet,USA,NY,40.754698,-73.992152,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.142689,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.052238,"[{'categories': [{'id': 13003, 'name': 'Bar', ...",0.307704,"[{'categories': [{'id': 13068, 'name': 'Americ...",0.005365,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005395
1,Solidea,USA,New York,40.715145,-74.005897,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.138882,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054807,"[{'categories': [{'id': 13010, 'name': 'Dive B...",0.271334,"[{'categories': [{'id': 13145, 'name': 'Fast F...",0.005311,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005433
2,Live2Support,USA,Long Island City,40.748876,-73.981579,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 11025, 'name': 'Child ...",0.137877,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054420,"[{'categories': [{'id': 13007, 'name': 'Beer G...",0.296729,"[{'categories': [{'id': 13072, 'name': 'Asian ...",0.005315,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005413
3,Petciety,USA,New York,40.753207,-73.995886,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 12057, 'name': 'Primar...",0.135554,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054378,"[{'categories': [{'id': 13006, 'name': 'Beer B...",0.282870,"[{'categories': [{'id': 13068, 'name': 'Americ...",0.005122,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005426
4,Cowrite,USA,New York,40.767924,-73.983602,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 12058, 'name': 'Elemen...",0.142138,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054061,"[{'categories': [{'id': 13011, 'name': 'Gay Ba...",0.300735,"[{'categories': [{'id': 13065, 'name': 'Restau...",0.005059,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005389


In [87]:
### Condition 9
top_offices = sc.add_score(top_offices, 9)

In [88]:
top_offices.head()

,name,country_code,city,latitude,longitude,condition1_results,condition1_score,condition3_results,condition3_score,condition5_results,...,condition4_results,condition4_score,condition6_results,condition6_score,condition7_results,condition7_score,condition8_results,condition8_score,condition9_results,condition9_score
0,Zoomabet,USA,NY,40.754698,-73.992152,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.052238,"[{'categories': [{'id': 13003, 'name': 'Bar', ...",0.307704,"[{'categories': [{'id': 13068, 'name': 'Americ...",0.005365,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005395,"[{'categories': [{'id': 11134, 'name': 'Pet Gr...",0.005198
1,Solidea,USA,New York,40.715145,-74.005897,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054807,"[{'categories': [{'id': 13010, 'name': 'Dive B...",0.271334,"[{'categories': [{'id': 13145, 'name': 'Fast F...",0.005311,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005433,"[{'categories': [{'id': 11134, 'name': 'Pet Gr...",0.005234
2,Live2Support,USA,Long Island City,40.748876,-73.981579,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054420,"[{'categories': [{'id': 13007, 'name': 'Beer G...",0.296729,"[{'categories': [{'id': 13072, 'name': 'Asian ...",0.005315,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005413,"[{'categories': [{'id': 11134, 'name': 'Pet Gr...",0.005233
3,Petciety,USA,New York,40.753207,-73.995886,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054378,"[{'categories': [{'id': 13006, 'name': 'Beer B...",0.282870,"[{'categories': [{'id': 13068, 'name': 'Americ...",0.005122,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005426,"[{'categories': [{'id': 11134, 'name': 'Pet Gr...",0.005097
4,Cowrite,USA,New York,40.767924,-73.983602,"[{'Unnamed: 0': 363, 'name': 'Hirshorn Zuckerm...",0.110497,"[{'Unnamed: 0': 1253, 'name': 'Neocleus', 'cou...",0.082706,"[{'Unnamed: 0': 3436, 'name': 'Morristown Muni...",...,"[{'categories': [{'id': 13035, 'name': 'Coffee...",0.054061,"[{'categories': [{'id': 13011, 'name': 'Gay Ba...",0.300735,"[{'categories': [{'id': 13065, 'name': 'Restau...",0.005059,"[{'categories': [{'id': 18008, 'name': 'Basket...",0.005389,"[{'categories': [{'id': 11134, 'name': 'Pet Gr...",0.005097


In [89]:
top_offices.to_csv('./data/topoffices2_score.csv')

### 5. Calculating total score and getting top 3 locations

In [90]:
top3_offices = total_score(top_offices)
top3_offices.reset_index(drop=True, inplace=True)

In [96]:
top3_offices[:3][["name", "country_code", "city", "total_score"]]

,name,country_code,city,total_score
0,iPharro Media,USA,New York,0.825351
1,Digital Railroad,USA,New York,0.824847
2,Struq,USA,NaN,0.824063


### 6. Plotting

In [105]:
import src.plot as pl
from src.criteria import conditions
loc1 = to_dic(top_offices, 0)
loc2 = to_dic(top_offices, 1)
loc3 = to_dic(top_offices, 2)

In [109]:
loc1_map = pl.plot_map(loc1, save=True)
loc2_map = pl.plot_map(loc2, save=True)
loc3_map = pl.plot_map(loc3, save=True)

## LOCATION 1 iPharro Media

In [110]:
loc1_map

## LOCATION 2 Digital Railroad

In [111]:
loc2_map

## LOCATION 3 Struq

In [113]:
loc3_map

In [108]:
## re-load py files
import importlib
import src.plot as pl
importlib.reload(pl)

<module 'src.plot' from 'C:\\Users\\Sbort\\ironhack\\project-III-geospatial-data\\src\\plot.py'>